# VectorStoreRetrieverMemory

`VectorStoreRetrieverMemory` lưu trữ bộ nhớ trong một vector store và truy vấn K tài liệu 'liên quan' nhất bất cứ khi nào được gọi.
Điều này khác với hầu hết các lớp bộ nhớ khác ở chỗ nó không theo dõi rõ ràng thứ tự của cuộc trò chuyện.

Trong hướng dẫn này, chúng ta sẽ khám phá ứng dụng thực tế của `VectorStoreRetrieverMemory` thông qua một kịch bản phỏng vấn mô phỏng. Thông qua ví dụ này, chúng ta sẽ thấy `VectorStoreRetrieverMemory` tìm kiếm thông tin dựa trên mức độ liên quan ngữ nghĩa thay vì thứ tự thời gian của các cuộc trò chuyện như thế nào.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Initializing the Vector Store

Next, we'll set up our vector store using FAISS. FAISS is an efficient similarity search library that will help us store and retrieve conversation embeddings:

In [3]:
import faiss
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores.faiss import FAISS

# Initialize OpenAI embeddings
embeddings_model = OpenAIEmbeddings()

# Create FAISS index
embedding_size = 1536  # Size for OpenAI embeddings
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

This setup creates an in-memory vector store that will maintain our conversation embeddings for quick similarity searches.

## Saving Interview Conversations

Now, we'll create our memory system and populate it with example interview conversations.

Note that by setting `k=1`, we ensure that only the single most relevant conversation is returned. (In real applications, you might want to increase this value to provide more context.):

In [4]:
from langchain.memory import VectorStoreRetrieverMemory

# We set k=1 to show that vector lookups still can return semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

# Save arbitrary conversations
memory.save_context(
    inputs={
        "human": "Hello, thank you for coming to the interview today. Please introduce yourself."
    },
    outputs={
        "ai": "Hello. I'm a junior developer who majored in Computer Science. In college, I mainly used Java and Python, and recently, I participated in a web development project where I gained experience developing services for real users."
    },
)
memory.save_context(
    inputs={"human": "What was your role in the project?"},
    outputs={
        "ai": "My role was as a backend developer. I was responsible for processing user data and developing server logic, implementing RESTful APIs for communication with the frontend. I also participated in database design."
    },
)
memory.save_context(
    inputs={
        "human": "If you faced any difficulties in team projects, how did you resolve them?"
    },
    outputs={
        "ai": "We had some communication issues at the beginning of the project. To resolve this, our team held regular meetings to share each person's progress. Also, when problems arose, we actively shared opinions and worked to find reasonable solutions."
    },
)
memory.save_context(
    inputs={"human": "What do you consider your strengths as a developer?"},
    outputs={
        "ai": "My strengths are quick learning ability and problem-solving skills. I can quickly acquire new technologies and tools, and when faced with complex problems, I can propose creative solutions. Also, I value teamwork and consider collaboration with colleagues important."
    },
)

/tmp/ipykernel_799818/480526100.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever)


## Retrieving Relevant Conversations

Let's see how the system retrieves relevant information based on queries:

In [5]:
# Query about educational background
print("Query: What was the interviewee's major?")
print(
    memory.load_memory_variables({"prompt": "What was the interviewee's major?"})[
        "history"
    ]
)

Query: What was the interviewee's major?
human: Hello, thank you for coming to the interview today. Please introduce yourself.
ai: Hello. I'm a junior developer who majored in Computer Science. In college, I mainly used Java and Python, and recently, I participated in a web development project where I gained experience developing services for real users.


In [6]:
# Query about project experience
print("Query: What was the interviewee's role in the project?")
print(
    memory.load_memory_variables(
        {"human": "What was the interviewee's role in the project?"}
    )["history"]
)

Query: What was the interviewee's role in the project?
human: What was your role in the project?
ai: My role was as a backend developer. I was responsible for processing user data and developing server logic, implementing RESTful APIs for communication with the frontend. I also participated in database design.


This approach is particularly valuable when building systems that need to reference past conversations contextually, such as in `customer service bots`, `educational assistants`, or any application where `maintaining context-aware conversation history` is important.